In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
!pip install evaluate
import evaluate 
import numpy as np 
from transformers import DataCollatorWithPadding

c:\Users\Korhan\anaconda3\envs\torch_generic_env\Lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Users\Korhan\anaconda3\envs\torch_generic_env\Lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please s


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
model_path = "google-bert/bert-base-uncased"

id2label = {
    0: "Anxiety", 
    1: "Depression", 
    2: "Stress", 
    3: "Suicidal",
    4: "Normal",
    5: "Bipolar",
    6: "Personality disorder",
}
label2id = {
    "Anxiety" : 0,
    "Depression" : 1,
    "Stress" : 2,
    "Suicidal": 3,
    "Normal" : 4,
    "Bipolar" : 5,
    "Personality disorder" : 6,
}

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=7, id2label=id2label, label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
for name, param in model.base_model.named_parameters():
    param.requires_grad = False

for name, param in model.base_model.named_parameters():
    if "pooler" in name:
        param.requires_grad = True

In [4]:
import json
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader


with open("ma_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

X_train = data['X_train']
y_train = data['y_train']

X_test = data['X_test']
y_test = data['y_test']

In [ ]:
import torch
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, label2id, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.label2id = label2id  
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx], 
            truncation=True, 
            padding='max_length', 
            max_length=self.max_length,
            return_tensors='pt'
        )
        item = {key: encoding[key].squeeze(0) for key in encoding}  
        
        # Etiketleri sayısal hale getiriyoruz
        label = self.label2id[self.labels[idx]]
        item['labels'] = torch.tensor(label, dtype=torch.long)
        
        return item

In [13]:
def preprocess_fn(data_X):
    return tokenizer(data_X, truncation=True, padding='max_length', max_length=512, return_tensors="pt")


train_dataset = TextDataset(X_train, y_train, tokenizer, label2id)
test_dataset = TextDataset(X_test, y_test, tokenizer, label2id)

In [ ]:
from transformers import DataCollatorWithPadding

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8)

# Data collator for dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Trainer arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

# Trainer
trainer = Trainer(
    model=model,                     # the instantiated  Transformers model to be trained
    args=training_args,              # training arguments, defined above
    train_dataset=train_dataset,     # training dataset
    eval_dataset=test_dataset,       # evaluation dataset
    data_collator=data_collator,     # data collator for padding
)

# Train the model
trainer.train()

comet_ml is installed but the Comet API Key is not configured. Please set the `COMET_API_KEY` environment variable to enable Comet logging. Check out the documentation for other ways of configuring it: https://www.comet.com/docs/v2/guides/experiment-management/configure-sdk/#set-the-api-key


  0%|          | 0/31827 [00:00<?, ?it/s]

{'loss': 1.9944, 'grad_norm': 6.1836066246032715, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}
{'loss': 1.9733, 'grad_norm': 8.926336288452148, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.0}
{'loss': 1.9691, 'grad_norm': 7.101653099060059, 'learning_rate': 3e-06, 'epoch': 0.0}
{'loss': 1.9761, 'grad_norm': 5.4672017097473145, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.0}
{'loss': 1.9122, 'grad_norm': 9.088139533996582, 'learning_rate': 5e-06, 'epoch': 0.0}
{'loss': 1.8992, 'grad_norm': 6.528930187225342, 'learning_rate': 6e-06, 'epoch': 0.01}
{'loss': 1.937, 'grad_norm': 5.9685750007629395, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.01}
{'loss': 1.8844, 'grad_norm': 4.932955265045166, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.01}
{'loss': 1.8117, 'grad_norm': 5.185669898986816, 'learning_rate': 9e-06, 'epoch': 0.01}
{'loss': 1.8359, 'grad_norm': 4.158491611480713, 'learning_rate': 1e-05, 'epoch': 0.01}
{'loss': 1.876, 'grad_norm': 6.1535949707

TrainOutput(global_step=31827, training_loss=0.907061036901252, metrics={'train_runtime': 7701.8094, 'train_samples_per_second': 33.058, 'train_steps_per_second': 4.132, 'total_flos': 6.69921344805888e+16, 'train_loss': 0.907061036901252, 'epoch': 3.0})

In [15]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(axis=-1)
    
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [16]:
trainer = Trainer(
    model=model,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

results = trainer.evaluate()
print("Evaluation Results : ", results)

C:\Users\Korhan\AppData\Local\Temp\ipykernel_15072\54315042.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
comet_ml is installed but the Comet API Key is not configured. Please set the `COMET_API_KEY` environment variable to enable Comet logging. Check out the documentation for other ways of configuring it: https://www.comet.com/docs/v2/guides/experiment-management/configure-sdk/#set-the-api-key


  0%|          | 0/2653 [00:00<?, ?it/s]

Evaluation Results :  {'eval_loss': 0.7978276014328003, 'eval_model_preparation_time': 0.013, 'eval_accuracy': 0.6930436421905929, 'eval_precision': 0.6856571139443369, 'eval_recall': 0.6930436421905929, 'eval_f1': 0.6858203543037106, 'eval_runtime': 557.1547, 'eval_samples_per_second': 38.083, 'eval_steps_per_second': 4.762}
